In [1]:
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
import pandas as pd

In [2]:
df_addresses = pd.read_csv("address_url_list.csv", names = ["url"], header = 0, index_col = 0)

In [3]:
# use headers to convince Compass.com scrape is not a bot

req_headers = {
        'accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language':'en-US,en;q=0.5',
        'referer':'https://www.google.com/',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode':'navigate',
        'sec-fetch-site':'cross-site',
        'sec-fetch-user':'?1',
        'te' : 'trailers',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:94.0) Gecko/20100101 Firefox/94.0'
    }

In [4]:
# Create dictionary to store scraped house details

house_dict = {"address":[],"zipcode":[],"sale_price":[],"beds":[],
                  "baths":[],"house_sqft":[],"lotsize":[], "year_built":[]}

for i, link in enumerate(df_addresses["url"]):
    response = requests.get(link, headers=req_headers)
    time.sleep(4)
    if (i+1 % 40 == 0):
        time.sleep(10)
    
    if (i+1 % 200 == 0):
        time.sleep(20)
        
    soup = BeautifulSoup(response.text, "lxml")

    address = soup.find_all("p", {"class": "summary__StyledAddress-e4c4ok-8 fWNtMX textIntent-title1"})[0]
    zipcode = soup.find_all("span", {"class": "summary__StyledAddressSubtitle-e4c4ok-9 leoAGJ"})[0]
    sale_price = soup.find_all("div", {"class": "textIntent-title2"})[0]
    beds = soup.find_all("div", {"class": "textIntent-title2"})[1]
    baths = soup.find_all("div", {"class": "textIntent-title2"})[2]
    house_sqft = soup.find_all("div", {"class": "textIntent-title2"})[3]
    lotsize = soup.find_all("span", {"class": "building-info__BuildingInfoLineItem-sc-85jvb8-1 fjpHBc"})[3]
    year_built = soup.find_all("span", {"class": "building-info__BuildingInfoLineItem-sc-85jvb8-1 fjpHBc"})[1]
    
    
    if address != None:
        house_dict["address"].append(address.text)
    else:
        house_dict["address"].append(np.nan)
    
    if zipcode != None:
        house_dict["zipcode"].append(re.findall('\d+', zipcode.text))
    else:
        house_dict["zipcode"].append(np.nan)
        
    if sale_price != None:
        house_dict["sale_price"].append(re.findall('[0-9,]+',sale_price.text))
    else:
        house_dict["sale_price"].append(np.nan)
    
    if beds != None:
        house_dict["beds"].append(beds.text)
    else:
        house_dict["beds"].append(np.nan)
        
    if baths != None:
        house_dict["baths"].append(baths.text)
    else:
        house_dict["baths"].append(np.nan)
        
    if house_sqft != None:
        house_dict["house_sqft"].append(re.findall('[0-9,]+', house_sqft.text))
    else:
        house_dict["house_sqft"].append(np.nan)
  
    if lotsize != None:
        house_dict["lotsize"].append(re.findall('\d+,\d+',lotsize.text))
    else:
        house_dict["lotsize"].append(np.nan)
        
    if year_built != None:
        house_dict["year_built"].append(re.findall('[0-9]+',year_built.text))
    else:
        house_dict["year_built"].append(np.nan)
        

house_dict

{'address': ['2934 South Bellaire Street',
  '1885 West Stoll Place',
  '5179 Clay Street',
  '1115 Cook Street',
  '3318 Gilpin Street',
  '1118 South Garfield Street',
  '1225 South Winona Court',
  '2215 South Marion Street',
  '1075 King Street',
  '1844 West 39th Avenue',
  '2811 Vrain Street',
  '4750 North Decatur Street',
  '801 Kearney Street',
  '21345 East 40th Avenue',
  '141 West 1st Avenue',
  '3324 Navajo Street',
  '1598 West 51st Avenue',
  '1877 Race Street',
  '343 Elati Street',
  '441 Wolff Street',
  '1342 South Clay Street',
  '720 South Clayton Street',
  '1687 South Winona Court',
  '2211 Holly Street',
  '318 South Corona Street',
  '9630 Fox Street',
  '3424 West Tennessee Avenue',
  '352 Delaware Street',
  '730 Fairfax Street',
  '850 South Gilpin Street',
  '746 Detroit Street',
  '3859 Bryant Street',
  '85 South Tamarac Street',
  '5534 East Custer Place',
  '161 South Jasmine Street',
  '2151 South Humboldt Street',
  '3020 York Street',
  '1968 Jasmine

In [5]:
df_houses = pd.DataFrame(house_dict)

In [6]:
df_houses.to_pickle("houses_dataframe.pkl")